In [1]:
cd

C:\Users\torye


C:\Users\torye\anaconda3\envs\GPU\lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
C:\Users\torye\anaconda3\envs\GPU\lib\site-packages\IPython\core\magics\osm.py:428: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import shutil
import csv

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from tensorflow.keras import layers, models, Input

#tf.config.set_visible_devices([], 'GPU')

# Attack Dataset Load

In [3]:
#df = pd.read_csv('./work/dataset/gear_dataset_hex.csv')
#df = pd.read_csv('./work/dataset/RPM_dataset_hex.csv')
#df = pd.read_csv('./work/dataset/DoS_dataset_hex.csv')
#df = pd.read_csv('./work/dataset/Fuzzy_dataset_hex.csv')


#df = pd.read_csv('./work/Survival_and_Challenge_2019/HYUNDAI_Sonata/HY_Sonata_Replay_HEX.csv')


df = pd.read_csv('./work/Survival_and_Challenge_2019/KIA_Soul/KIA_Soul_Replay_HEX.csv')




#df = df.iloc[:1000]

protocol = "CAN"
#protocol = "CAN-FD"

#attack = "gear"
#attack = "RPM"
#attack = "DoS"
#attack = "Fuzzy"
attack = "Replay"

#attack = "Malfunction"
#attack = "Flooding"
#attack = "Fuzzing"

# Make folder

In [4]:
#version = "HY_SONATA"
version = "KIA_SOUL"
#version = "CHEVROLET_SPARK"
#version = "HY_G80"

folder_name = f"DCNN_Sliding_{protocol}_{version}"

directory = "./Models/22.DCNN_(VC)"  # Replace with your desired path
path = os.path.join(directory, folder_name)

"""
# Check that the folder already exists and Delete folder if it exists
if os.path.exists(path):
    shutil.rmtree(path) 

# Create a new folder
os.makedirs(path)
"""

# Preprocessing : ID Sequence

In [5]:
#======================================# 
print("Start")
#======================================# 

F = 28
df = df.drop(['Timestamp'], axis=1)
df['Arbitration_ID_binary'] = df['Arbitration_ID'].apply(lambda x: bin(int(x, 16))[2:].zfill(29))
np_id = [np.array([int(char) for char in bin_str]) for bin_str in df['Arbitration_ID_binary'].astype(str)]


ID_Sequence = []

for i in range(df.shape[0] - F):
    concatenated_rows = np.concatenate([np_id[i + j] for j in range(F + 1)], axis=0)
    ID_Sequence.append(concatenated_rows)


# For training_ID
training_id = np.array(ID_Sequence)
reshaped_array = training_id.reshape(-1, 29, 29, 1)



print(reshaped_array.shape)

Start
(566310, 29, 29, 1)


# Labelling

In [6]:
labeling = df['Class']
labeling = labeling.shift(-F)
labeling = labeling.drop(range(labeling.shape[0] - F, labeling.shape[0])).astype(int)
labeling = np.array(labeling)


num_classes = 2
one_hot_labels = to_categorical(labeling, num_classes=num_classes)
one_hot_labels_int = one_hot_labels.astype(int)

# Dataset Split (7:3)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(reshaped_array, one_hot_labels_int, test_size=0.3, random_state=42)

# DCNN

In [8]:
#======================================#  
#   Reduced_Inception_ResNet
#======================================#  

def Conv_2D_Block(x, model_width, kernel, strides=(1, 1), padding="same"):
    # 2D Convolutional Block with BatchNormalization
    x = tf.keras.layers.Conv2D(model_width, kernel, strides=strides, padding=padding, kernel_initializer="he_normal")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    return x


def classifier(inputs, class_number):
    # Construct the Classifier Group
    # inputs       : input vector
    # class_number : number of output classes
    out = tf.keras.layers.Dense(class_number, activation='softmax')(inputs)
    return out


def regressor(inputs, feature_number):
    # Construct the Regressor Group
    # inputs         : input vector
    # feature_number : number of output features
    out = tf.keras.layers.Dense(feature_number, activation='linear')(inputs)
    return out


def Inception_ResNet_Module_A(inputs, filterB1_1, filterB2_1, filterB2_2, filterB3_1, filterB3_2, filterB3_3, filterB4_1, i):
    # Inception ResNet Module A - Block i
    branch1x1 = Conv_2D_Block(inputs, filterB1_1, (1, 1))

    branch3x3 = Conv_2D_Block(inputs, filterB2_1, (1, 1))
    branch3x3 = Conv_2D_Block(branch3x3, filterB2_2, (3, 3))

    branch3x3dbl = Conv_2D_Block(inputs, filterB3_1, (1, 1))
    branch3x3dbl = Conv_2D_Block(branch3x3dbl, filterB3_2, (3, 3))
    branch3x3dbl = Conv_2D_Block(branch3x3dbl, filterB3_3, (3, 3))

    branch_concat = tf.keras.layers.concatenate([branch1x1, branch3x3, branch3x3dbl], axis=-1)
    branch1x1_ln = tf.keras.layers.Conv2D(filterB4_1, (1, 1), activation='linear', strides=(1, 1), padding='same', kernel_initializer="he_normal")(branch_concat)

    x = tf.keras.layers.Add(name='Inception_ResNet_Block_A'+str(i))([inputs, branch1x1_ln])
    x = tf.keras.layers.BatchNormalization()(x)
    out = tf.keras.layers.Activation('relu')(x)

    return out


def Inception_ResNet_Module_B(inputs, filterB1_1, filterB2_1, filterB2_2, filterB2_3, filterB3_1, i):
    # Inception ResNet Module B - Block i
    branch1x1 = Conv_2D_Block(inputs, filterB1_1, (1, 1))

    branch7x7 = Conv_2D_Block(inputs, filterB2_1, (1, 1))
    branch7x7 = Conv_2D_Block(branch7x7, filterB2_2, (1, 3))
    branch7x7 = Conv_2D_Block(branch7x7, filterB2_3, (3, 1))

    branch_concat = tf.keras.layers.concatenate([branch1x1, branch7x7], axis=-1)
    branch1x1_ln = tf.keras.layers.Conv2D(filterB3_1, (1, 1), activation='linear', strides=(1, 1), padding='same', kernel_initializer="he_normal")(branch_concat)

    x = tf.keras.layers.Add(name='Inception_ResNet_Block_B'+str(i))([inputs, branch1x1_ln])
    x = tf.keras.layers.BatchNormalization()(x)
    out = tf.keras.layers.Activation('relu')(x)

    return out

"""
def Inception_ResNet_Module_C(inputs, filterB1_1, filterB2_1, filterB2_2, filterB2_3, filterB3_1, i):
    # Inception ResNet Module C - Block i
    branch1x1 = Conv_2D_Block(inputs, filterB1_1, (1, 1))

    branch3x3 = Conv_2D_Block(inputs, filterB2_1, (1, 1))
    branch3x3 = Conv_2D_Block(branch3x3, filterB2_2, (1, 3))
    branch3x3 = Conv_2D_Block(branch3x3, filterB2_3, (3, 1))

    branch_concat = tf.keras.layers.concatenate([branch1x1, branch3x3], axis=-1)
    branch1x1_ln = tf.keras.layers.Conv2D(filterB3_1, (1, 1), activation='linear', strides=(1, 1), padding='same', kernel_initializer="he_normal")(branch_concat)

    x = tf.keras.layers.Add(name='Inception_ResNet_Block_C'+str(i))([inputs, branch1x1_ln])
    x = tf.keras.layers.BatchNormalization()(x)
    out = tf.keras.layers.Activation('relu')(x)

    return out
"""

def Reduction_Block_A(inputs, filterB1_1, filterB2_1, filterB2_2, filterB2_3):
    # Reduction Block A
    branch_pool = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(inputs)

    branch3x3 = Conv_2D_Block(inputs, filterB1_1, 3, strides=(2, 2), padding='valid')

    branch3x3dbl = Conv_2D_Block(inputs, filterB2_1, (1, 1))
    branch3x3dbl = Conv_2D_Block(branch3x3dbl, filterB2_2, (3, 3))
    branch3x3dbl = Conv_2D_Block(branch3x3dbl, filterB2_3, (3, 3), strides=(2, 2), padding='valid')

    x = tf.keras.layers.concatenate([branch_pool, branch3x3, branch3x3dbl], axis=-1, name='Reduction_Block_A')
    x = tf.keras.layers.BatchNormalization()(x)
    out = tf.keras.layers.Activation('relu')(x)

    return out


def Reduction_Block_B(inputs, filterB1_1, filterB1_2, filterB2_1, filterB2_2, filterB3_1, filterB3_2, filterB3_3):
    # Reduction Block B
    branch_pool = tf.keras.layers.MaxPooling2D(pool_size=(3, 3))(inputs)

    branch3x3 = Conv_2D_Block(inputs, filterB1_1, (1, 1))
    branch3x3 = Conv_2D_Block(branch3x3, filterB1_2, (3, 3), strides=(2, 2), padding='valid')

    branch3x3_2 = Conv_2D_Block(inputs, filterB2_1, (1, 1))
    branch3x3_2 = Conv_2D_Block(branch3x3_2, filterB2_2, (3, 3), strides=(2, 2), padding='valid')

    branch3x3dbl = Conv_2D_Block(inputs, filterB3_1, (1, 1))
    branch3x3dbl = Conv_2D_Block(branch3x3dbl, filterB3_2, (3, 3))
    branch3x3dbl = Conv_2D_Block(branch3x3dbl, filterB3_3, (3, 3), strides=(2, 2), padding='valid')

    x = tf.keras.layers.concatenate([branch_pool, branch3x3, branch3x3_2, branch3x3dbl], axis=-1)
    x = tf.keras.layers.BatchNormalization()(x)
    out = tf.keras.layers.Activation('relu')(x)

    return out


class Inception_ResNet:
    def __init__(self, length, width, num_channel, num_filters, problem_type='Classification',
                 output_nums=10, pooling='avg', dropout_rate=False, auxilliary_outputs=False):
        # length: Input Image Length (x-dim)
        # width: Input Image Width (y-dim) [Normally same as the x-dim i.e., Square shape]
        # model_depth: Depth of the Model
        # model_width: Width of the Model
        # kernel_size: Kernel or Filter Size of the Input Convolutional Layer
        # num_channel: Number of Channels of the Input Predictor Signals
        # problem_type: Regression or Classification
        # output_nums: Number of Output Classes in Classification mode and output features in Regression mode
        # pooling: Choose either 'max' for MaxPooling or 'avg' for Averagepooling
        # dropout_rate: If turned on, some layers will be dropped out randomly based on the selected proportion
        # auxilliary_outputs: Two extra Auxullary outputs for the Inception models, acting like Deep Supervision
        self.length = length
        self.width = width
        self.num_channel = num_channel
        self.num_filters = num_filters
        self.problem_type = problem_type
        self.output_nums = output_nums
        self.pooling = pooling
        self.dropout_rate = dropout_rate
        self.auxilliary_outputs = auxilliary_outputs

    def MLP(self, x):
        if self.pooling == 'avg':
            x = tf.keras.layers.GlobalAveragePooling2D()(x)
        elif self.pooling == 'max':
            x = tf.keras.layers.GlobalMaxPooling2D()(x)
        x = tf.keras.layers.Flatten()(x)
        if self.dropout_rate:
            x = tf.keras.layers.Dropout(self.dropout_rate)(x)
        outputs = tf.keras.layers.Dense(self.output_nums, activation='linear')(x)
        if self.problem_type == 'Classification':
            outputs = tf.keras.layers.Dense(self.output_nums, activation='softmax')(x)

        return outputs

    def Inception_ResNet_v1(self):
        inputs = tf.keras.Input((self.length, self.width, self.num_channel))  # The input tensor
        # Stem
        x = Conv_2D_Block(inputs, 32, 3)
        x = Conv_2D_Block(x, 32, 3, padding='valid')
      # x = Conv_2D_Block(x, 64, 3)
        x = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
        x = Conv_2D_Block(x, 64, 1)
        x = Conv_2D_Block(x, 128, 3)
      # x = Conv_2D_Block(x, 128, 3, strides=(2, 2), padding='valid')
        x = Conv_2D_Block(x, 128, 3)

        # 5x Inception ResNet A Blocks - 35 x 35 x 256
        for i in range(1):
            x = Inception_ResNet_Module_A(x, 32, 32, 32, 32, 32, 32, 128, i)

        aux_output_0 = []
        if self.auxilliary_outputs:
            # Auxilliary Output 0
            aux_pool = tf.keras.layers.AveragePooling2D(pool_size=(5, 5), strides=(3, 3))(x)
            aux_conv = Conv_2D_Block(aux_pool, 128, (1, 1))
            aux_conv = Conv_2D_Block(aux_conv, 768, (5, 5), padding='valid')
            aux_output_0 = self.MLP(aux_conv)

        x = Reduction_Block_A(x, 192, 96, 96, 128)  # Reduction Block A: 17 x 17 x 768

        # 10x Inception ResNet B Blocks - 17 x 17 x 768
        for i in range(1):
            x = Inception_ResNet_Module_B(x, 64, 64, 64, 64, 448, i)

        aux_output_1 = []
        if self.auxilliary_outputs:
            # Auxilliary Output 1
            aux_pool = tf.keras.layers.AveragePooling2D(pool_size=(5, 5), strides=(3, 3), padding='valid')(x)
            aux_conv = Conv_2D_Block(aux_pool, 128, (1, 1))
            aux_conv = Conv_2D_Block(aux_conv, 768, (5, 5), padding='valid')
            aux_output_1 = self.MLP(aux_conv)

        x = Reduction_Block_B(x, 128, 192, 128, 128, 128, 128, 128)  # Reduction Block B: 8 x 8 x 1280

        """
        # 5x Inception ResNet C Blocks - 8 x 8 x 1280
        for i in range(5):
            x = Inception_ResNet_Module_C(x, 128, 192, 192, 192, 1792, i)
        """


        # Final Dense MLP Layer for the outputs
        final_output = self.MLP(x)
        # Create model.
        model = tf.keras.Model(inputs, final_output, name='Inception_v4')
        if self.auxilliary_outputs:
            model = tf.keras.Model(inputs, outputs=[final_output, aux_output_0, aux_output_1], name='Inception_ResNet_v1')

        return model



model = Inception_ResNet(length=29, width=29, num_channel=1, num_filters=64, problem_type='Classification', output_nums=2, pooling='avg', dropout_rate=0.2).Inception_ResNet_v1()


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "Inception_v4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 29, 29, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 29, 29, 32)   320         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 29, 29, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 29, 29, 32)   0           ['batch_normalization[

In [29]:
# Define a custom callback to log batch losses
class BatchLossCSVLogger(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs=None):
        self.losses.append(logs.get('loss'))

    def on_epoch_end(self, epoch, logs=None):
        self.val_losses.append(logs.get('val_loss'))

# Create an instance of the custom callback
batch_loss_logger = BatchLossCSVLogger()


history = model.fit(X_train, y_train, epochs=10,
                    batch_size=64,
                    verbose=2,
                    callbacks=[batch_loss_logger] )

Epoch 1/10
6195/6195 - 96s - loss: 0.0140 - accuracy: 0.9948 - 96s/epoch - 15ms/step
Epoch 2/10
6195/6195 - 98s - loss: 0.0138 - accuracy: 0.9951 - 98s/epoch - 16ms/step
Epoch 3/10
6195/6195 - 98s - loss: 0.0137 - accuracy: 0.9949 - 98s/epoch - 16ms/step
Epoch 4/10
6195/6195 - 98s - loss: 0.0128 - accuracy: 0.9953 - 98s/epoch - 16ms/step
Epoch 5/10
6195/6195 - 97s - loss: 0.0127 - accuracy: 0.9955 - 97s/epoch - 16ms/step
Epoch 6/10
6195/6195 - 101s - loss: 0.0128 - accuracy: 0.9955 - 101s/epoch - 16ms/step
Epoch 7/10
6195/6195 - 98s - loss: 0.0121 - accuracy: 0.9957 - 98s/epoch - 16ms/step
Epoch 8/10
6195/6195 - 97s - loss: 0.0114 - accuracy: 0.9959 - 97s/epoch - 16ms/step
Epoch 9/10
6195/6195 - 97s - loss: 0.0116 - accuracy: 0.9959 - 97s/epoch - 16ms/step
Epoch 10/10
6195/6195 - 98s - loss: 0.0113 - accuracy: 0.9960 - 98s/epoch - 16ms/step


In [30]:
# Write batch losses to a CSV file with a dynamic filename
file_path = os.path.join(path, f'{attack}_batch_losses.csv')
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Batch', 'Loss'])
    for i, loss in enumerate(batch_loss_logger.losses):
        writer.writerow([i, loss])
        
print(f"Batch validation losses saved to {file_path}")

# Save epoch validation losses to CSV
csv_file_path = os.path.join(path, f'{attack}_epoch_val_losses.csv')
with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Epoch', 'Val_Loss'])
    for i, val_loss in enumerate(batch_loss_logger.val_losses):
        writer.writerow([i, val_loss])

print(f"Epoch validation losses saved to {csv_file_path}")

Batch validation losses saved to ./Models/22.DCNN_(VC)\DCNN_Sliding_CAN_KIA_SOUL\Replay_batch_losses.csv
Epoch validation losses saved to ./Models/22.DCNN_(VC)\DCNN_Sliding_CAN_KIA_SOUL\Replay_epoch_val_losses.csv


# Model Test

In [31]:
# Perform prediction
y_pred = model.predict(X_test)

# Select the class with the highest probability as the predicted value
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Convert the confusion matrix to a DataFrame and save it as a CSV file
conf_matrix_df = pd.DataFrame(conf_matrix)
conf_matrix_file_name = f"{path}/{attack}_confusion_matrix.csv"
conf_matrix_df.to_csv(conf_matrix_file_name, index=False)
print(f"Confusion matrix saved to {conf_matrix_file_name}")

# Generate a classification report
class_report = classification_report(y_true, y_pred_classes, digits=4, output_dict=True)


class_report_df = pd.DataFrame(class_report).transpose()
class_report_file_name = f"{path}/{attack}_classification_report.csv"
class_report_df.to_csv(class_report_file_name, index=True)
print(f"Classification report saved to {class_report_file_name}")

5310/5310 [==============================] - 23s 4ms/step
Confusion matrix saved to ./Models/22.DCNN_(VC)\DCNN_Sliding_CAN_KIA_SOUL/Replay_confusion_matrix.csv
Classification report saved to ./Models/22.DCNN_(VC)\DCNN_Sliding_CAN_KIA_SOUL/Replay_classification_report.csv


In [32]:
conf_matrix_df

,0,1
0,119050,4317
1,4521,42005


In [33]:
# Extract True Positive, False Positive, False Negative, and True Negative from the confusion matrix
# Assumption: Binary classification problem (0 and 1)
TP = conf_matrix_df.iloc[1, 1]  # True Positive
FP = conf_matrix_df.iloc[0, 1]  # False Positive
FN = conf_matrix_df.iloc[1, 0]  # False Negative
TN = conf_matrix_df.iloc[0, 0]  # True Negative

# Calculate evaluation metrics
accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Print the results
print(f"TP: {TP}")
print(f"FP: {FP}")
print(f"FN: {FN}")
print(f"TN: {TN}")

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

TP: 42005
FP: 4317
FN: 4521
TN: 119050
Accuracy: 0.9480
Precision: 0.9068
Recall: 0.9028
F1 Score: 0.9048
